In [1]:
import os
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
from torchvision import transforms
import numpy as np

# Set random seed for reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

class SegmentationDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = transforms.Resize((256, 256))(mask)
            mask = transforms.ToTensor()(mask)

        return image, mask

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
])

val_test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load dataset paths
image_dir = "/kaggle/input/kvasir-seg/Kvasir-SEG/images"
mask_dir = "/kaggle/input/kvasir-seg/Kvasir-SEG/masks"

images = sorted(os.listdir(image_dir))
masks = sorted(os.listdir(mask_dir))

image_paths = [os.path.join(image_dir, img) for img in images]
mask_paths = [os.path.join(mask_dir, mask) for mask in masks]

# Split into train (85%), validation (10%), and test (5%)
train_imgs, temp_imgs, train_masks, temp_masks = train_test_split(image_paths, mask_paths, test_size=0.15, random_state=42)
val_imgs, test_imgs, val_masks, test_masks = train_test_split(temp_imgs, temp_masks, test_size=0.3333, random_state=42)

# Initialize datasets
train_dataset = SegmentationDataset(train_imgs, train_masks, transform=train_transform)
val_dataset = SegmentationDataset(val_imgs, val_masks, transform=val_test_transform)
test_dataset = SegmentationDataset(test_imgs, test_masks, transform=val_test_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- Residual ConvBlock ---
class ResidualConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.0):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(dropout),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
        )
        self.relu = nn.ReLU(inplace=True)
        self.skip = nn.Conv2d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()

    def forward(self, x):
        residual = self.skip(x)
        out = self.conv(x)
        return self.relu(out + residual)

# --- Attention Block ---
class AttentionBlock(nn.Module):
    def __init__(self, f_g, f_l, f_int):
        super().__init__()
        self.W_g = nn.Sequential(nn.Conv2d(f_g, f_int, 1), nn.BatchNorm2d(f_int))
        self.W_x = nn.Sequential(nn.Conv2d(f_l, f_int, 1), nn.BatchNorm2d(f_int))
        self.psi = nn.Sequential(nn.Conv2d(f_int, 1, 1), nn.BatchNorm2d(1), nn.Sigmoid())
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        return x * self.psi(self.relu(self.W_g(g) + self.W_x(x)))

# --- CBAM ---
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.shared_MLP = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.shared_MLP(self.avg_pool(x)) + self.shared_MLP(self.max_pool(x)))

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        return self.sigmoid(self.conv(torch.cat([avg_out, max_out], dim=1)))

class CBAM(nn.Module):
    def __init__(self, channels, reduction=8, spatial_kernel=7):
        super().__init__()
        self.ca = ChannelAttention(channels, reduction)
        self.sa = SpatialAttention(spatial_kernel)

    def forward(self, x):
        return x * self.ca(x) * self.sa(x)

# --- Up + Conv ---
class UpConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.up(x)

# --- Morphological BRM ---
class MorphologicalGradientBRM(nn.Module):
    def __init__(self, kernel_size=3):
        super().__init__()
        padding = kernel_size // 2
        self.dilate = nn.MaxPool2d(kernel_size, stride=1, padding=padding)
        self.erode = nn.MaxPool2d(kernel_size, stride=1, padding=padding)

    def forward(self, x):
        dilation = self.dilate(x)
        erosion = -self.erode(-x)
        boundary = dilation - erosion
        return x + boundary

# --- Edge Enhancement Module ---
class EdgeEnhancement(nn.Module):
    def __init__(self):
        super().__init__()
        # Laplacian filter
        laplacian_kernel = torch.tensor([[0,  1,  0],
                                         [1, -4,  1],
                                         [0,  1,  0]], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        self.register_buffer('laplacian_kernel', laplacian_kernel)

        # Sobel filters
        sobel_x = torch.tensor([[-1, 0, 1], 
                                [-2, 0, 2], 
                                [-1, 0, 1]], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        sobel_y = torch.tensor([[-1, -2, -1], 
                                [0,  0,  0], 
                                [1,  2,  1]], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        self.register_buffer('sobel_x', sobel_x)
        self.register_buffer('sobel_y', sobel_y)

    def forward(self, x):
        laplacian = F.conv2d(x, self.laplacian_kernel, padding=1)
        sobel_x = F.conv2d(x, self.sobel_x, padding=1)
        sobel_y = F.conv2d(x, self.sobel_y, padding=1)
        
        sobel = torch.sqrt(sobel_x ** 2 + sobel_y ** 2)  # Gradient magnitude
        edge_enhanced = x + laplacian + sobel  # Combine edges with original feature maps
        
        return edge_enhanced


# --- Boundary Attention Module ---
class BoundaryAttentionModule(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.att = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 2, 1),
            nn.BatchNorm2d(in_channels // 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // 2, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.att(x)

# --- Attention UNet++ Full Boundary-Aware ---
class AttentionUNetPlusPlus(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, feature_scale=1, dropout=0.0):
        super().__init__()

        filters = [int(x / feature_scale) for x in [64, 128, 256, 512, 1024]]
        self.pool = nn.MaxPool2d(2, 2)

        # Encoder
        self.encoder1 = ResidualConvBlock(in_channels, filters[0], dropout)
        self.encoder2 = ResidualConvBlock(filters[0], filters[1], dropout)
        self.encoder3 = ResidualConvBlock(filters[1], filters[2], dropout)
        self.encoder4 = ResidualConvBlock(filters[2], filters[3], dropout)
        self.center = ResidualConvBlock(filters[3], filters[4], dropout)

        # Decoder
        self.up4 = UpConv(filters[4], filters[3])
        self.att4 = AttentionBlock(filters[3], filters[3], filters[2])
        self.dec4 = ResidualConvBlock(filters[4], filters[3], dropout)
        self.cbam4 = CBAM(filters[3])
        self.bam4 = BoundaryAttentionModule(filters[3])

        self.up3 = UpConv(filters[3], filters[2])
        self.att3 = AttentionBlock(filters[2], filters[2], filters[1])
        self.dec3 = ResidualConvBlock(filters[3], filters[2], dropout)
        self.cbam3 = CBAM(filters[2])
        self.bam3 = BoundaryAttentionModule(filters[2])

        self.up2 = UpConv(filters[2], filters[1])
        self.att2 = AttentionBlock(filters[1], filters[1], filters[0])
        self.dec2 = ResidualConvBlock(filters[2], filters[1], dropout)
        self.cbam2 = CBAM(filters[1])
        self.bam2 = BoundaryAttentionModule(filters[1])

        self.up1 = UpConv(filters[1], filters[0])
        self.att1 = AttentionBlock(filters[0], filters[0], filters[0] // 2)
        self.dec1 = ResidualConvBlock(filters[1], filters[0], dropout)
        self.cbam1 = CBAM(filters[0])
        self.bam1 = BoundaryAttentionModule(filters[0])

        # Output layers
        self.out1 = nn.Conv2d(filters[0], out_channels, 1)
        self.out2 = nn.Conv2d(filters[1], out_channels, 1)
        self.out3 = nn.Conv2d(filters[2], out_channels, 1)
        self.out4 = nn.Conv2d(filters[3], out_channels, 1)

        # Extra modules
        self.brm = MorphologicalGradientBRM()
        self.edge_enhance = EdgeEnhancement()

        # Multi-scale input fusion
        self.input_down1 = nn.AvgPool2d(2)
        self.input_down2 = nn.AvgPool2d(4)
        self.input_down3 = nn.AvgPool2d(8)

    def forward(self, x):
        # Input fusion
        x1 = x
        x2 = F.interpolate(self.input_down1(x), size=x.shape[2:], mode='bilinear', align_corners=True)
        x3 = F.interpolate(self.input_down2(x), size=x.shape[2:], mode='bilinear', align_corners=True)
        x4 = F.interpolate(self.input_down3(x), size=x.shape[2:], mode='bilinear', align_corners=True)
        x = (x1 + x2 + x3 + x4) / 4

        e1 = self.encoder1(x)
        e2 = self.encoder2(self.pool(e1))
        e3 = self.encoder3(self.pool(e2))
        e4 = self.encoder4(self.pool(e3))
        center = self.center(self.pool(e4))

        d4 = self.up4(center)
        e4 = self.att4(d4, e4)
        d4 = self.dec4(torch.cat([d4, e4], dim=1))
        d4 = self.bam4(self.cbam4(d4))

        d3 = self.up3(d4)
        e3 = self.att3(d3, e3)
        d3 = self.dec3(torch.cat([d3, e3], dim=1))
        d3 = self.bam3(self.cbam3(d3))

        d2 = self.up2(d3)
        e2 = self.att2(d2, e2)
        d2 = self.dec2(torch.cat([d2, e2], dim=1))
        d2 = self.bam2(self.cbam2(d2))

        d1 = self.up1(d2)
        e1 = self.att1(d1, e1)
        d1 = self.dec1(torch.cat([d1, e1], dim=1))
        d1 = self.bam1(self.cbam1(d1))

        out_main = self.out1(d1)
        out_main = self.brm(out_main)
        out_main = self.edge_enhance(out_main)

        out_aux1 = F.interpolate(self.out2(d2), size=out_main.shape[2:], mode='bilinear', align_corners=True)
        out_aux2 = F.interpolate(self.out3(d3), size=out_main.shape[2:], mode='bilinear', align_corners=True)
        out_aux3 = F.interpolate(self.out4(d4), size=out_main.shape[2:], mode='bilinear', align_corners=True)

        return out_main, out_aux1, out_aux2, out_aux3

# --- Test the model ---
if __name__ == "__main__":
    model = AttentionUNetPlusPlus(in_channels=1, out_channels=1, dropout=0.1)
    x = torch.randn(1, 1, 256, 256)
    outputs = model(x)
    for i, out in enumerate(outputs):
        print(f"Output {i+1} shape: {out.shape}")


Output 1 shape: torch.Size([1, 1, 256, 256])
Output 2 shape: torch.Size([1, 1, 256, 256])
Output 3 shape: torch.Size([1, 1, 256, 256])
Output 4 shape: torch.Size([1, 1, 256, 256])


In [3]:

epochs = 500
lr = 1e-4

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

def visualize_predictions(loader, title, save_path=None):
    model.eval()
    with torch.no_grad():
        inputs, targets = next(iter(loader))
        inputs, targets = inputs.to(device), targets.to(device)

        raw_outputs = model(inputs)
        if isinstance(raw_outputs, tuple):
            main_output = raw_outputs[0]
        else:
            main_output = raw_outputs

        outputs = (torch.sigmoid(main_output) > 0.5).float()

        fig, axes = plt.subplots(3, 5, figsize=(15, 9))
        for i in range(5):
            img = inputs[i].cpu().permute(1, 2, 0) if inputs.shape[1] == 3 else inputs[i, 0].cpu()
            axes[0, i].imshow(img, cmap='gray' if inputs.shape[1] == 1 else None)
            axes[1, i].imshow(targets[i].cpu().squeeze(), cmap='gray')
            axes[2, i].imshow(outputs[i].cpu().squeeze(), cmap='gray')
            for ax in axes[:, i]:
                ax.axis("off")

        plt.suptitle(title)
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
        plt.show()


# --- Hyperparameters ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# --- Model, Optimizer ---
model = AttentionUNetPlusPlus(in_channels=3, out_channels=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import os

# --- Dice Loss ---
def dice_loss(pred, target, smooth=1.):
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum(dim=(2, 3))
    union = pred.sum(dim=(2, 3)) + target.sum(dim=(2, 3))
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice.mean()

# --- Combined BCE + Dice Loss ---
def combined_loss(preds, target):
    bce = nn.BCEWithLogitsLoss()
    weights = [0.6, 0.2, 0.1, 0.1]
    total_loss = 0
    for i, pred in enumerate(preds):
        loss_bce = bce(pred, target)
        loss_dice = dice_loss(pred, target)
        total = loss_bce + loss_dice
        total_loss += weights[i] * total
    return total_loss

# --- Metrics ---
def calc_metrics(pred, target):
    pred = torch.sigmoid(pred) > 0.5
    target = target.bool()
    intersection = (pred & target).float().sum((1, 2, 3))
    union = (pred | target).float().sum((1, 2, 3))
    iou = (intersection / (union + 1e-6)).mean()
    dice = (2 * intersection / (pred.sum((1, 2, 3)) + target.sum((1, 2, 3)) + 1e-6)).mean()
    accuracy = (pred == target).float().mean()
    return dice.item(), iou.item(), accuracy.item()

# --- TensorBoard Writer ---
writer = SummaryWriter(log_dir="runs/AttentionUNetPlusPlus")

# --- Training Loop ---
best_dice = 0.0
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("visualizations", exist_ok=True)

for epoch in range(epochs):
    model.train()
    train_loss, train_acc = 0, 0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}") as pbar:
        for img, mask in pbar:
            img, mask = img.to(device), mask.to(device)
            optimizer.zero_grad()
            outputs = model(img)
            loss = combined_loss(outputs, mask)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, _, acc = calc_metrics(outputs[0], mask)
            train_acc += acc
            pbar.set_postfix({"loss": loss.item(), "accuracy": acc})
    
    # --- Validation ---
    model.eval()
    val_loss, dice_total, iou_total, acc_total = 0, 0, 0, 0
    with torch.no_grad():
        for img, mask in val_loader:
            img, mask = img.to(device), mask.to(device)
            outputs = model(img)
            loss = combined_loss(outputs, mask)
            val_loss += loss.item()
            dice, iou, acc = calc_metrics(outputs[0], mask)
            dice_total += dice
            iou_total += iou
            acc_total += acc

    avg_train_loss = train_loss / len(train_loader)
    avg_train_acc = train_acc / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    avg_dice = dice_total / len(val_loader)
    avg_iou = iou_total / len(val_loader)
    avg_acc = acc_total / len(val_loader)

    print(f"\nEpoch {epoch+1}: "
          f"Train Loss={avg_train_loss:.4f} | Train Acc={avg_train_acc:.4f} | "
          f"Val Loss={avg_val_loss:.4f} | Dice={avg_dice:.4f} | IoU={avg_iou:.4f} | Accuracy={avg_acc:.4f}\n")

    # --- TensorBoard Logging ---
    writer.add_scalar("Loss/Train", avg_train_loss, epoch)
    writer.add_scalar("Loss/Validation", avg_val_loss, epoch)
    writer.add_scalar("Metrics/Dice", avg_dice, epoch)
    writer.add_scalar("Metrics/IoU", avg_iou, epoch)
    writer.add_scalar("Metrics/Accuracy", avg_acc, epoch)

    # --- Save Best Model ---
    if avg_dice > best_dice:
        best_dice = avg_dice
        ckpt_path = f"checkpoints/best_model_epoch{epoch+1}_dice{avg_dice:.4f}.pth"
        torch.save(model.state_dict(), ckpt_path)
        print(f"✅ Best model saved: {ckpt_path}")

writer.close()


Epoch 1/500: 100%|██████████| 107/107 [01:14<00:00,  1.44it/s, loss=1.18, accuracy=0.777]



Epoch 1: Train Loss=1.3615 | Train Acc=0.5883 | Val Loss=1.2614 | Dice=0.3881 | IoU=0.2634 | Accuracy=0.6783

✅ Best model saved: checkpoints/best_model_epoch1_dice0.3881.pth


Epoch 2/500: 100%|██████████| 107/107 [01:04<00:00,  1.67it/s, loss=1.09, accuracy=0.784]



Epoch 2: Train Loss=1.2081 | Train Acc=0.7798 | Val Loss=1.1372 | Dice=0.3537 | IoU=0.2399 | Accuracy=0.8295



Epoch 3/500: 100%|██████████| 107/107 [01:04<00:00,  1.66it/s, loss=1.02, accuracy=0.888]


...


Epoch 498: Train Loss=0.3403 | Train Acc=0.9640 | Val Loss=0.6050 | Dice=0.7323 | IoU=0.6076 | Accuracy=0.9294



Epoch 499/500: 100%|██████████| 107/107 [01:04<00:00,  1.66it/s, loss=0.461, accuracy=0.972]



Epoch 499: Train Loss=0.3403 | Train Acc=0.9642 | Val Loss=0.5902 | Dice=0.7413 | IoU=0.6188 | Accuracy=0.9292



Epoch 500/500: 100%|██████████| 107/107 [01:04<00:00,  1.66it/s, loss=0.407, accuracy=0.949]



Epoch 500: Train Loss=0.3363 | Train Acc=0.9646 | Val Loss=0.5739 | Dice=0.7441 | IoU=0.6206 | Accuracy=0.9324

